In [53]:
# Imports
import os
from dotenv import load_dotenv

# wiki 
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.retrievers import WikipediaRetriever

# for loading web page
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import ChatOpenAI
from langsmith import Client


from langchain_core.tools import tool
from langchain.agents import create_agent

In [25]:
load_dotenv('../.env')

# constants
UNITTEST_MOCK_URL = 'https://docs.python.org/3/library/unittest.mock.html'

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGCHAIN_API_KEY')

langsmith_client = Client(api_key=LANGSMITH_API_KEY)

In [12]:
wiki_retriever = WikipediaRetriever(wiki_client=WikipediaAPIWrapper(), top_k_results=1, lang='en')
wiki = WikipediaQueryRun(api_wrapper=wiki_retriever)

In [13]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaRetriever(wiki_client=<module 'wikipedia' from '/home/bitcot/python/courses/env/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1))

In [42]:
web_loader = WebBaseLoader(UNITTEST_MOCK_URL)
web_content = web_loader.load()
web_content_split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(web_content)
vector_db = FAISS.from_documents(web_content_split, OpenAIEmbeddings(api_key=OPENAI_API_KEY))
web_retriever = vector_db.as_retriever()
web_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7838b80aff50>, search_kwargs={})

In [54]:
@tool
def search_python_docs(query: str) -> str:
    """Search Python unittest.mock documentation for relevant information."""
    docs = web_retriever.invoke(query)
    return "\n\n".join([doc.page_content for doc in docs])

In [55]:
tools = [wiki, search_python_docs]

In [43]:
opneai_llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-4o-mini')
opneai_llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x78387482b8f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x783874852360>, root_client=<openai.OpenAI object at 0x78387b1c2c90>, root_async_client=<openai.AsyncOpenAI object at 0x783874829dc0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [56]:
# Create agent using the new create_agent function
system_prompt = """You are a helpful assistant that can search Wikipedia and Python documentation.
When a user asks about Python testing or mock objects, use the search_python_docs tool.
For general knowledge questions, use the wikipedia tool."""

In [57]:
agent = create_agent(
    opneai_llm, 
    tools=tools,
    system_prompt=system_prompt
)

In [60]:
# Run the agent
inputs = {"messages": [{"role": "user", "content": "What is India?"}]}

In [61]:
# Stream the response
for chunk in agent.stream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What is India?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_DkO2vd8TCW9nletGHzHHY8r9)
 Call ID: call_DkO2vd8TCW9nletGHzHHY8r9
  Args:
    query: India
================================= Tool Message =================================
Name: wikipedia

Page: India
Summary: India, officially the Republic of India, is a country in South Asia.  It is the seventh-largest country by area; the most populous country since 2023; and, since its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime 